In [ ]:
!pip3 install python-chess

In [ ]:
!pip install patool

In [ ]:
import chess
import chess.engine
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import patoolib
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#example fen for testing
fen = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [ ]:
'''
Matric representation order: black, white:  king, queen, rook, knight, bishop, pawn
'''
def bitMap(fen):
  board = chess.Board(fen)
  orderMapping = {
    'k': 0,
    'K': 6,
    'q': 1,
    'Q': 7,
    'r': 2,
    'R': 8,
    'n': 3,
    'N': 9,
    'b': 4,
    'B': 10,
    'p': 5,
    'P': 11
  }
  material = {
    'k': 0,
    'K': 0,
    'q': -9,
    'Q': 9,
    'r': -5,
    'R': 5,
    'n': -3,
    'N': 3,
    'b': -3,
    'B': 3,
    'p': -1,
    'P': 1
  }
  alphabet = 'abcdefgh'
  bitMap = np.zeros((14, 8, 8))
  positions, turn, castlingRights, enPeasant, half1, half2 = fen.split(' ')
  row, column = [0, 0]
  whiteMaterial, blackMaterial = 0, 0
  for letter in positions:
    if letter.isnumeric():
      column += int(letter)
    elif letter == '/':
      row += 1
      column = 0
    else:
      peiceIndex = orderMapping[letter]
      if letter.isupper():
        bitMap[peiceIndex, row, column] = 1
      else:
        bitMap[peiceIndex, row, column] = -1
      if letter.isupper():
        whiteMaterial += material[letter]
      else:
        blackMaterial += material[letter]
      column += 1

  #showing the model the possible squares we can move to, which is known to help the model a lot
  first, second = 12, 13
  if not board.turn:
    first, second = 13, 12

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[first, row, column] = 1
  board.turn = not board.turn #looking at the possible moves for the person whos turn its not

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[second, row, column] = 1
  #since we originally placed the pieces by defining rank 0 as the eigth index, we have to flip this list to stay in accordance to the definition
  bitMap[12] = bitMap[12][::-1]
  bitMap[13] = bitMap[13][::-1]
  return bitMap, whiteMaterial, blackMaterial


#trying a different board representation to see if it performs better (after testing, it does. The current model uses this function as opposed to bitMap(fen).)
def compressedBitMap(fen):
  board = chess.Board(fen)
  orderMapping = {
    'k': 0,
    'K': 0,
    'q': 1,
    'Q': 1,
    'r': 2,
    'R': 2,
    'n': 3,
    'N': 3,
    'b': 4,
    'B': 4,
    'p': 5,
    'P': 5
  }
  material = {
    'k': 0,
    'K': 0,
    'q': -9,
    'Q': 9,
    'r': -5,
    'R': 5,
    'n': -3,
    'N': 3,
    'b': -3,
    'B': 3,
    'p': -1,
    'P': 1
  }
  alphabet = 'abcdefgh'
  bitMap = np.zeros((8, 8, 8))
  positions, turn, castlingRights, enPeasant, half1, half2 = fen.split(' ')
  row, column = [0, 0]
  whiteMaterial, blackMaterial = 0, 0
  for letter in positions:
    if letter.isnumeric():
      column += int(letter)
    elif letter == '/':
      row += 1
      column = 0
    else:
      peiceIndex = orderMapping[letter]
      if letter.isupper():
        bitMap[peiceIndex, row, column] = 1
      else:
        bitMap[peiceIndex, row, column] = -1
      if letter.isupper():
        whiteMaterial += material[letter]
      else:
        blackMaterial += material[letter]
      column += 1

  #showing the model the possible squares we can move to, which is known to help the model a lot
  first, second = 6, 7
  if not board.turn:
    first, second = 7, 6

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[first, row, column] = 1
  board.turn = not board.turn #looking at the possible moves for the person whos turn its not

  for move in board.legal_moves:
    column = alphabet.index(str(move)[2])
    row = int(str(move)[3]) - 1
    bitMap[second, row, column] = 1
  #since we originally placed the pieces by defining rank 0 as the eigth index, we have to flip this list to stay in accordance to the definition
  bitMap[6] = bitMap[6][::-1]
  bitMap[7] = bitMap[7][::-1]
  return bitMap, whiteMaterial, blackMaterial

Building the dataset

In [ ]:
patoolib.extract_archive('ChessEvals.zip')
dataframe = pd.read_csv('ChessEvals/chessData.csv')

INFO patool: Extracting ChessEvals.zip ...
INFO:patool:Extracting ChessEvals.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_xrp8mdcg -- ChessEvals.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_xrp8mdcg -- ChessEvals.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... ChessEvals.zip extracted to `ChessEvals2' (multiple files in root).
INFO:patool:... ChessEvals.zip extracted to `ChessEvals2' (multiple files in root).


In [ ]:
def createTraining(amount, df):
  x = []
  y = []
  for index in df.index[:amount]:
    row = df.iloc[index]
    map, whiteMats, blackMats = compressedBitMap(row['FEN'])
    matsDiff = whiteMats + blackMats
    evalString = row['Evaluation']
    if '#' in evalString:
      try:
        eval = int(evalString[1:])
      except:
        continue
    else:
      try:
        eval = int(evalString)
      except:
        continue
    evalAdjusted = (eval / 100) + matsDiff
    x.append(map)
    y.append(evalAdjusted)
  x = np.array(x)
  x = np.reshape(x, (x.shape[0], 1, 8, 8, 8))
  y = np.array(y)
  y = np.reshape(y, (x.shape[0],))
  return x, y

Building the model

In [ ]:
model = tf.keras.models.Sequential()

model.add( layers.Input(shape=(1, 8, 8, 8)) )
model.add( layers.Conv2D(48, kernel_size = 3, activation = 'relu', padding = 'same',input_shape = (1, 8, 8, 8)) )
model.add( layers.BatchNormalization() )
model.add( layers.Conv2D(48, kernel_size = 3, activation = 'relu', padding = 'same') )
model.add( layers.BatchNormalization() )
model.add( layers.Conv2D(32, kernel_size = 3, activation = 'relu', padding = 'same') )
model.add( layers.BatchNormalization() )

model.add( layers.Flatten() )
model.add( layers.Dense(32, activation = 'relu') )
model.add( layers.Dense(32, activation = 'relu') )
model.add( layers.Dense(1) )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 8, 8, 48)       3504      
                                                                 
 batch_normalization (Batch  (None, 1, 8, 8, 48)       192       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 8, 8, 48)       20784     
                                                                 
 batch_normalization_1 (Bat  (None, 1, 8, 8, 48)       192       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 8, 8, 32)       13856     
                                                                 
 batch_normalization_2 (Bat  (None, 1, 8, 8, 32)       1

In [ ]:
x, y = createTraining(200000, dataframe)
model.compile(optimizer = 'adam',
              loss = 'mse',
              metrics = ['mse'])
model.fit(x, y, epochs = 50, batch_size= 512)
'''
The model is currently overfitting a litte. If I were to come back to this, I would increase the training examples and decrease the number of epochs. I may also play with the number of
trainable parameters in the model to see if it is a limiting factor/causing the model to overfit.
'''

Epoch 1/50
391/391 [==============================] - 167s 416ms/step - loss: 33.3315 - mse: 33.3315
Epoch 2/50
391/391 [==============================] - 146s 373ms/step - loss: 24.0097 - mse: 24.0097
Epoch 3/50
391/391 [==============================] - 110s 282ms/step - loss: 18.3817 - mse: 18.3817
Epoch 4/50
391/391 [==============================] - 110s 281ms/step - loss: 14.6151 - mse: 14.6151
Epoch 5/50
391/391 [==============================] - 108s 277ms/step - loss: 12.2304 - mse: 12.2304
Epoch 6/50
391/391 [==============================] - 110s 281ms/step - loss: 10.4442 - mse: 10.4442
Epoch 7/50
391/391 [==============================] - 160s 410ms/step - loss: 9.1851 - mse: 9.1851
Epoch 8/50
391/391 [==============================] - 159s 407ms/step - loss: 7.9884 - mse: 7.9884
Epoch 9/50
391/391 [==============================] - 120s 306ms/step - loss: 7.1587 - mse: 7.1587
Epoch 10/50
391/391 [==============================] - 109s 279ms/step - loss: 6.7625 - mse: 6.76

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save('/content/Chess.h5')